##### 가장 기초적인 방법의 선형회귀

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 재사용을 위해 랜덤값을 초기화합니다.
torch.manual_seed(2023)    

In [2]:
x_train = torch.FloatTensor([[-1,3],[3,7],[4,9]])
y_train = torch.FloatTensor([[3],[6],[9]])

In [3]:
W = torch.zeros(size=(2,1), requires_grad=True)
b = torch.zeros(size=(1,), requires_grad=True)
W, b

(tensor([[0.],
         [0.]], requires_grad=True),
 tensor([0.], requires_grad=True))

In [4]:
hypothesis = torch.matmul(x_train, W) + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


In [5]:
cost = torch.mean((hypothesis - y_train) ** 2)
optimizer = optim.SGD([W, b], lr=0.01)

In [6]:
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    hypothesis = torch.matmul(x_train, W) + b
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x)개선
    optimizer.zero_grad()

    if not (epoch % 100) or (epoch % 100 == 99):
        print()
        print(f"Epoch:{epoch:4d},                   W_0:{W[0].item():.6f},  W_1:{W[1].item():.6f},  b:{b.item():.6f}")
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그출력
    if not (epoch % 100) or (epoch % 100 == 99):
        print(f"Epoch:{epoch:4d},  Cost:{cost.item():9.6f},  W_0:{W[0].item():.6f},  W_1:{W[1].item():.6f},  b:{b.item():.6f}")        


Epoch:   0,                   W_0:0.000000,  W_1:0.000000,  b:0.000000
Epoch:   0,  Cost:42.000000,  W_0:0.340000,  W_1:0.880000,  b:0.120000

Epoch:  99,                   W_0:0.004593,  W_1:0.927964,  b:0.143060
Epoch:  99,  Cost: 0.222988,  W_0:0.004008,  W_1:0.928276,  b:0.142473

Epoch: 100,                   W_0:0.004008,  W_1:0.928276,  b:0.142473
Epoch: 100,  Cost: 0.222909,  W_0:0.003435,  W_1:0.928584,  b:0.141882

Epoch: 199,                   W_0:-0.026794,  W_1:0.949597,  b:0.075048
Epoch: 199,  Cost: 0.216787,  W_0:-0.027007,  W_1:0.949777,  b:0.074337

Epoch: 200,                   W_0:-0.027007,  W_1:0.949777,  b:0.074337
Epoch: 200,  Cost: 0.216728,  W_0:-0.027219,  W_1:0.949957,  b:0.073625

Epoch: 299,                   W_0:-0.046907,  W_1:0.967128,  b:0.004012
Epoch: 299,  Cost: 0.211029,  W_0:-0.047104,  W_1:0.967301,  b:0.003305

Epoch: 300,                   W_0:-0.047104,  W_1:0.967301,  b:0.003305
Epoch: 300,  Cost: 0.210972,  W_0:-0.047300,  W_1:0.967474,  b:

In [7]:
test_var = torch.FloatTensor([[5, 12]])
with torch.no_grad():
    pred_y = torch.matmul(test_var, W) + b
    print(f"훈련 후 테스트 값이 [5, 12]일 떄의 예측 값 : {pred_y.item():.3f}")

훈련 후 테스트 값이 [5, 12]일 떄의 예측 값 : 11.634


In [8]:
hypothesis = torch.matmul(x_train, torch.zeros(size=(2,1), requires_grad=True)) + torch.zeros(size=(1,), requires_grad=True)
mse = torch.mean((hypothesis - y_train) ** 2).item()
print(mse)

42.0


In [9]:
print(optimizer.state)

defaultdict(<class 'dict'>, {tensor([[-0.1779],
        [ 1.0828]], requires_grad=True): {'momentum_buffer': None}, tensor([-0.4696], requires_grad=True): {'momentum_buffer': None}})


#### nn.Module로 구현하는 선형회귀

In [10]:
x_train = torch.FloatTensor([[-1,3],[3,7],[4,9]])
y_train = torch.FloatTensor([[3],[6],[9]])

In [11]:
input_dim, output_dim = 2, 1

model = nn.Linear(in_features=input_dim, out_features=output_dim)
optimizer = optim.SGD(params=model.parameters(), lr=0.01)

In [12]:
for param in model.parameters():
    print(param)    
    print()

Parameter containing:
tensor([[-0.1004,  0.3112]], requires_grad=True)

Parameter containing:
tensor([0.6338], requires_grad=True)



In [13]:
nb_epochs = 1000
for epoch in range(nb_epochs):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)     # PyTorch에서 제공하는 mse
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step() 
    
    W_0 = list(model.parameters())[0][0][0]   
    W_1 = list(model.parameters())[0][0][1]
    b   = list(model.parameters())[1][0]
    # 100번마다 로그출력
    if not (epoch % 100) or (epoch % 100 == 99):
        print(f"Epoch:{epoch:4d},  Cost:{cost.item():10.6f},  W_0:{W_0.item():.6f},  W_1:{W_1.item():.6f},  b:{b.item():.6f}")           

Epoch:   0,  Cost: 16.516907,  W_0:0.119613,  W_1:0.858689,  b:0.705695
Epoch:  99,  Cost:  0.262315,  W_0:0.118300,  W_1:0.820457,  b:0.607471
Epoch: 100,  Cost:  0.262244,  W_0:0.118131,  W_1:0.820632,  b:0.606665
Epoch: 199,  Cost:  0.255344,  W_0:0.098107,  W_1:0.839062,  b:0.528644
Epoch: 200,  Cost:  0.255275,  W_0:0.097894,  W_1:0.839251,  b:0.527866
Epoch: 299,  Cost:  0.248562,  W_0:0.076796,  W_1:0.857905,  b:0.451373
Epoch: 300,  Cost:  0.248495,  W_0:0.076584,  W_1:0.858092,  b:0.450606
Epoch: 399,  Cost:  0.241961,  W_0:0.055719,  W_1:0.876514,  b:0.375154
Epoch: 400,  Cost:  0.241895,  W_0:0.055510,  W_1:0.876699,  b:0.374397
Epoch: 499,  Cost:  0.235535,  W_0:0.034922,  W_1:0.894876,  b:0.299955
Epoch: 500,  Cost:  0.235471,  W_0:0.034715,  W_1:0.895058,  b:0.299208
Epoch: 599,  Cost:  0.229279,  W_0:0.014402,  W_1:0.912992,  b:0.225761
Epoch: 600,  Cost:  0.229218,  W_0:0.014199,  W_1:0.913171,  b:0.225024
Epoch: 699,  Cost:  0.223190,  W_0:-0.005843,  W_1:0.930865,  b:

In [15]:
new_var = torch.FloatTensor([[5, 12]])
pred_y = model(new_var)
print(f"훈련 후 입력이 [5,12]일 때의 예측값 : {pred_y.item():.3f}")

훈련 후 입력이 [5,12]일 때의 예측값 : 11.411


#### nn.Module class로 구현하는 선형회귀

In [16]:
x_train = torch.FloatTensor([[-1,3],[3,7],[4,9]])
y_train = torch.FloatTensor([[3],[6],[9]])

In [17]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(in_features=2, out_features=1)
        
    def forward(self, x):
        return self.linear(x)    # forward 연산

In [18]:
model = LinearRegressionModel()
optimizer = optim.SGD(params=model.parameters(), lr=0.01)

In [19]:
nb_epochs = 2000
for epoch in range(nb_epochs):
    prediction = model(x_train)
    cost = F.mse_loss(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    model_params = list(model.parameters())
    W_0, W_1, b = model_params[0][0][0], model_params[0][0][1], model_params[1][0]
    if not(epoch % 100) or (epoch % 100 == 99):
        print(f"Epoch : {epoch:4d},  Cost : {cost.item():10.6f},  W_0 : {W_0.item():.3f},  W_1 : {W_1.item():.3f},  b : {b.item():.3f}")

Epoch :    0,  Cost :  30.524385,  W_0 : 0.267,  W_1 : 0.807,  b : 0.794
Epoch :   99,  Cost :   0.273449,  W_0 : 0.156,  W_1 : 0.789,  b : 0.729
Epoch :  100,  Cost :   0.273374,  W_0 : 0.156,  W_1 : 0.789,  b : 0.728
Epoch :  199,  Cost :   0.266173,  W_0 : 0.132,  W_1 : 0.809,  b : 0.650
Epoch :  200,  Cost :   0.266101,  W_0 : 0.132,  W_1 : 0.810,  b : 0.649
Epoch :  299,  Cost :   0.259103,  W_0 : 0.110,  W_1 : 0.829,  b : 0.571
Epoch :  300,  Cost :   0.259033,  W_0 : 0.110,  W_1 : 0.829,  b : 0.570
Epoch :  399,  Cost :   0.252222,  W_0 : 0.088,  W_1 : 0.848,  b : 0.493
Epoch :  400,  Cost :   0.252154,  W_0 : 0.088,  W_1 : 0.848,  b : 0.492
Epoch :  499,  Cost :   0.245523,  W_0 : 0.067,  W_1 : 0.866,  b : 0.416
Epoch :  500,  Cost :   0.245457,  W_0 : 0.067,  W_1 : 0.867,  b : 0.416
Epoch :  599,  Cost :   0.239003,  W_0 : 0.046,  W_1 : 0.885,  b : 0.341
Epoch :  600,  Cost :   0.238939,  W_0 : 0.046,  W_1 : 0.885,  b : 0.340
Epoch :  699,  Cost :   0.232655,  W_0 : 0.026,  W_

In [20]:
new_var = torch.FloatTensor([[5, 12]])
with torch.no_grad():
    pred_y = model(new_var)
    print(f"훈련 후 입력이 [5, 12]일 때의 예측값 : {pred_y.item():.3f}")

훈련 후 입력이 [5, 12]일 때의 예측값 : 11.716
